In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
import random

# === Thay đổi đường dẫn dataset gốc của bạn ===
base_dir = "/kaggle/input/fisheye8k/Fisheye8K"
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "test")

# === Thư mục output ===
output_dir = "/kaggle/working/dataset_split"
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")

for d in ["train", "val"]:
    os.makedirs(os.path.join(images_dir, d), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, d), exist_ok=True)

# === Lấy toàn bộ ảnh từ cả train + val ===
all_images = []
for d in [train_dir, val_dir]:
    img_path = os.path.join(d, "images")
    for f in os.listdir(img_path):
        if f.endswith((".jpg", ".png", ".jpeg")):
            all_images.append(os.path.join(img_path, f))

print("Tổng số ảnh ban đầu:", len(all_images))

# === Shuffle để random ===
random.shuffle(all_images)

# === Chia 80/20 ===
split_idx = int(0.8 * len(all_images))
train_images = all_images[:split_idx]
val_images   = all_images[split_idx:]

def copy_files(image_list, subset):
    for img_path in image_list:
        file_name = os.path.basename(img_path)
        label_name = os.path.splitext(file_name)[0] + ".txt"

        # copy ảnh
        shutil.copy(img_path, os.path.join(images_dir, subset, file_name))

        # copy nhãn (nếu có)
        label_path = img_path.replace("images", "labels")
        label_path = os.path.splitext(label_path)[0] + ".txt"
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(labels_dir, subset, label_name))

copy_files(train_images, "train")
copy_files(val_images, "val")

print(f"Train: {len(train_images)} ảnh")
print(f"Val: {len(val_images)} ảnh")


In [ ]:
import textwrap

# Đường dẫn để lưu file YAML
# Thay tên data.yaml
data_yaml_path = '/kaggle/working/data.yaml'

yaml_content = textwrap.dedent("""
    train: /kaggle/working/dataset_split/images/train
    val: /kaggle/working/dataset_split/images/val
    nc: 5
    names: ["Bus", "Bike", "Car", "Pedestrian", "Truck"]
""")

# Chỉ cần một lần ghi file duy nhất
with open(data_yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"✅ Đã tạo file {data_yaml_path} thành công.")

In [ ]:
from ultralytics import YOLO
import os

# ==============================
# 🚀 STAGE 1: Training từ đầu
# ==============================

print("🚀 Bắt đầu huấn luyện với dữ liệu kết hợp và cấu hình tối ưu...")
model = YOLO('yolov8m.pt')

results = model.train(
    data='/kaggle/working/data.yaml',
    epochs=20,
    imgsz=1280,
    batch=8,

    # Training strategy
    lr0=0.005, lrf=0.01,
    weight_decay=0.0005,
    patience=10,

    # Augmentation
    degrees=10,
    translate=0.1,
    scale=0.7,
    perspective=0.0003,
    mosaic=1.0,
    mixup=0.15,
    hsv_h=0.02, hsv_s=0.7, hsv_v=0.4,

    # Inference settings
    conf=0.25,
    iou=0.5,

    # Output
    project='/kaggle/working/yolov8m_training_results_v2',
    name='stage1'
)

In [ ]:
from ultralytics import YOLO
import os

# 🚀 Resume Stage 1 từ checkpoint cuối cùng (last.pt)
model = YOLO('/kaggle/working/yolov8m_training_results_v2/stage1/weights/last.pt')

print("🚀 Tiếp tục huấn luyện Stage 1 từ last.pt...")
results = model.train(
    resume=True
)

print("🎉 Quá trình huấn luyện Stage 1 đã được resume thành công!")

In [ ]:
# ==============================
# 🚀 STAGE 2: Fine-tune từ Best
# ==============================
print("🔥 Bắt đầu Stage 2 Fine-tuning từ best.pt...")

best_model_path = "/kaggle/working/yolov8m_training_results_v2/stage1/weights/best.pt"
model = YOLO(best_model_path)

results = model.train(
    data='/kaggle/working/data.yaml',
    epochs=30,
    imgsz=1280,
    batch=8,

    # Training strategy (LR giảm để fine-tune ổn định)
    lr0=0.001,
    lrf=0.005,
    weight_decay=0.0005,
    patience=10,

     # Augmentation (giảm nhẹ so với Stage 1)
    degrees=5,         # xoay ít hơn
    translate=0.05,    # dịch chuyển ít hơn
    scale=0.5,         # thu phóng vừa phải
    perspective=0.0002,
    mosaic=0.5,        # giảm mosaic để ảnh tự nhiên hơn
    mixup=0.05,        # giảm mixup, tránh ảnh bị quá nhiễu
    hsv_h=0.02,
    hsv_s=0.5,         # giảm saturation
    hsv_v=0.3,         # giảm độ sáng


    # Inference settings
    conf=0.25,
    iou=0.5,

    # Output
    project='/kaggle/working/yolov8m_training_results_v2',
    name='stage2'
)

print("🎉 Stage 2 huấn luyện đã hoàn tất!")

In [ ]:
from ultralytics import YOLO

# 1. THAY ĐỔI ĐƯỜNG DẪN NÀY cho đúng với file của bạn
path_to_last_checkpoint = '/kaggle/working/yolov8m_training_results_v2/stage2/weights/last.pt'

# 2. Tải mô hình trực tiếp từ checkpoint cuối cùng
model = YOLO(path_to_last_checkpoint)

# 3. Chạy lệnh resume với compile=True
print("🚀 Bắt đầu tiếp tục quá trình huấn luyện Fisheye...")
results = model.train(
    resume=True,
)

print("🎉 Quá trình huấn luyện đã hoàn tất!")


In [ ]:
import json
import os
import torch
import numpy as np
import random
from ultralytics import YOLO
from tqdm import tqdm
from PIL import Image # Thêm thư viện để xử lý và lưu ảnh

# --- BƯỚC 1: CẤU HÌNH ---
best_model_path = '/kaggle/working/yolov8m_training_results_v2/stage2/weights/best.pt'
test_images_folder = '/kaggle/working/dataset_split/images/val'
ground_truth_json_path = '/kaggle/input/fisheye8k/Fisheye8K/test/test.json'
# --- THAY ĐỔI: Thêm đường dẫn để lưu ảnh kết quả ---
output_visualization_folder = '/kaggle/working/prediction_021025'

IOU_THRESHOLD = 0.7
CONFIDENCE_THRESHOLD = 0.25

# --- BƯỚC 2: HÀM HỖ TRỢ ---
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def load_ground_truth(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    images_map = {img['id']: img['file_name'] for img in data['images']}
    ground_truths = {}
    total_gt_objects = 0
    for ann in data['annotations']:
        total_gt_objects += 1
        filename = images_map[ann['image_id']]
        if filename not in ground_truths:
            ground_truths[filename] = []
        bbox = ann['bbox']
        gt_box = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
        ground_truths[filename].append({'class_id': ann['category_id'], 'bbox': gt_box})
    return ground_truths, total_gt_objects

# --- BƯỚC 3: THỰC THI ---
# Tải model
print("🚀 Đang tải model...")
model = YOLO(best_model_path)

# Đọc dữ liệu ground truth
print("📚 Đang đọc dữ liệu Ground Truth từ test.json...")
ground_truths, total_ground_truth_objects = load_ground_truth(ground_truth_json_path)

# Lấy toàn bộ ảnh test
print("📝 Lấy toàn bộ ảnh từ thư mục test...")
all_image_files = [os.path.join(test_images_folder, f) for f in os.listdir(test_images_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Chạy dự đoán trên toàn bộ tập ảnh
print(f"🔮 Bắt đầu chạy dự đoán trên toàn bộ {len(all_image_files)} ảnh...")
results_list = []
for img_path in tqdm(all_image_files, desc="Đang dự đoán"):
    result = model.predict(source=img_path, conf=CONFIDENCE_THRESHOLD, verbose=False)
    results_list.append(result[0])

# --- THAY ĐỔI: LƯU 10 ẢNH KẾT QUẢ NGẪU NHIÊN ---
print("\n🖼️ Đang lưu 10 ảnh kết quả ngẫu nhiên...")
# Tạo thư mục nếu chưa tồn tại
os.makedirs(output_visualization_folder, exist_ok=True)
# Chọn ngẫu nhiên 10 kết quả
random_results_to_show = random.sample(results_list, min(10, len(results_list)))
for result in random_results_to_show:
    # result.plot() trả về ảnh dưới dạng mảng numpy (BGR)
    plotted_image_bgr = result.plot()
    # Chuyển từ BGR sang RGB để PIL có thể xử lý đúng màu
    plotted_image_rgb = plotted_image_bgr[..., ::-1]
    # Tạo đối tượng ảnh từ mảng
    img_to_save = Image.fromarray(plotted_image_rgb)
    # Lưu ảnh
    output_path = os.path.join(output_visualization_folder, os.path.basename(result.path))
    img_to_save.save(output_path)
print(f"✅ Đã lưu ảnh vào thư mục: {output_visualization_folder}")

# Khởi tạo các biến đếm
total_true_positives = 0
total_false_positives = 0

print("\n🔍 Bắt đầu so khớp dự đoán và Ground Truth...")
# Vòng lặp so khớp
for result in tqdm(results_list, desc="Đang xử lý ảnh"):
    filename = os.path.basename(result.path)
    pred_boxes = result.boxes.xyxy.cpu().numpy()
    pred_classes = result.boxes.cls.cpu().numpy().astype(int)
    gt_objects = ground_truths.get(filename, [])

    if not gt_objects:
        total_false_positives += len(pred_boxes)
        continue

    gt_boxes_for_img = [item['bbox'] for item in gt_objects]
    gt_classes_for_img = [item['class_id'] for item in gt_objects]
    gt_matched = [False] * len(gt_boxes_for_img)

    # Sắp xếp các dự đoán theo độ tin cậy giảm dần
    sorted_indices = np.argsort(-result.boxes.conf.cpu().numpy())

    for i in sorted_indices:
        pred_box = pred_boxes[i]
        pred_class = pred_classes[i]
        best_iou = 0
        best_gt_idx = -1

        for j in range(len(gt_boxes_for_img)):
            if gt_classes_for_img[j] == pred_class and not gt_matched[j]:
                iou = calculate_iou(pred_box, gt_boxes_for_img[j])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = j

        if best_iou >= IOU_THRESHOLD:
            gt_matched[best_gt_idx] = True
            total_true_positives += 1
        else:
            total_false_positives += 1

# --- BƯỚC 4: TÍNH TOÁN VÀ IN KẾT QUẢ ---
total_false_negatives = total_ground_truth_objects - total_true_positives

try:
    precision = total_true_positives / (total_true_positives + total_false_positives)
except ZeroDivisionError:
    precision = 0.0

try:
    recall = total_true_positives / (total_true_positives + total_false_negatives)
except ZeroDivisionError:
    recall = 0.0

try:
    f1_score = 2 * (precision * recall) / (precision + recall)
except ZeroDivisionError:
    f1_score = 0.0

print("\n--- KẾT QUẢ ĐÁNH GIÁ TRÊN TOÀN BỘ TẬP TEST ---")
print(f"Tổng số vật thể có thật: {total_ground_truth_objects}")
print("------------------------------------")
print(f"True Positives:    {total_true_positives}")
print(f"False Positives:   {total_false_positives}")
print(f"False Negatives:    {total_false_negatives}")
print("------------------------------------")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

In [ ]:
import json
from pathlib import Path
from ultralytics import YOLO
from tqdm.notebook import tqdm
import gc

# ==============================================================================
# 1. CẤU HÌNH
# ==============================================================================
CONFIG = {
    "project_root": "/kaggle/working/results",
    "test_image_dirs": [
        "/kaggle/input/fisheye1keval/images/images",
        "/kaggle/input/fisheye1keval/images1/images1"
    ],
    "model_for_prediction": "/kaggle/working/yolov8m_training_results_v2/stage2/weights/best.pt",
    "confidence_threshold": 0.25,
    "img_size": 1280,
    "batch_size": 100
}

# ==============================================================================
# 2. HÀM CHỨC NĂNG
# ==============================================================================
def get_image_id(img_name: str) -> int | None:
    try:
        img_name = img_name.split('.png')[0]
        sceneList = ['M', 'A', 'E', 'N']
        cameraIndx = int(img_name.split('_')[0].split('camera')[1])
        sceneIndx = sceneList.index(img_name.split('_')[1])
        frameIndx = int(img_name.split('_')[2])
        imageId = int(str(cameraIndx) + str(sceneIndx) + str(frameIndx))
        return imageId
    except (IndexError, ValueError):
        return None

CATEGORY_MAP = {0:0, 1:1, 2:2, 3:3, 4:4}

# ==============================================================================
# 3. KHỐI ĐIỀU KHIỂN CHÍNH
# ==============================================================================
if __name__ == "__main__":

    PROJECT_ROOT = Path(CONFIG["project_root"])
    TEMP_OUTPUT_FILE = PROJECT_ROOT / "temp_results.jsonl"
    SUBMISSION_FILE_PATH = PROJECT_ROOT / "submission.json"

    # --- A. Kiểm tra tiến trình cũ ---
    processed_image_ids = set()
    if TEMP_OUTPUT_FILE.exists():
        with open(TEMP_OUTPUT_FILE, 'r') as f:
            for line in f:
                try:
                    processed_image_ids.add(json.loads(line)['image_id'])
                except json.JSONDecodeError:
                    continue
        print(f"🔍 Đã tìm thấy file tạm. Bỏ qua {len(processed_image_ids)} ảnh đã xử lý.")

    # --- B. Gom và lọc ảnh ---
    all_test_images = [p for folder in CONFIG["test_image_dirs"] for p in Path(folder).glob("*.png")]
    images_to_process = [p for p in all_test_images if get_image_id(p.name) not in processed_image_ids]

    if not images_to_process:
        print("✅ Tất cả ảnh đã được xử lý.")
    else:
        print(f"🔥 Cần xử lý {len(images_to_process)} ảnh mới.")
        model = YOLO(CONFIG["model_for_prediction"])

        batch_size = CONFIG["batch_size"]
        with open(TEMP_OUTPUT_FILE, 'a') as f:
            for i in tqdm(range(0, len(images_to_process), batch_size), desc="Đang xử lý các lô"):
                batch_images = images_to_process[i : i + batch_size]
                if not batch_images:
                    continue

                results = model.predict(
                    source=[str(p) for p in batch_images],
                    stream=False,
                    imgsz=CONFIG["img_size"],
                    conf=CONFIG["confidence_threshold"],
                    iou=0.5,
                    max_det=1000,
                    device=0
                )

                for r in results:
                    image_id = get_image_id(Path(r.path).name)
                    if image_id is None:
                        continue
                    for box in r.boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        width, height = float(x2 - x1), float(y2 - y1)
                        category_id = CATEGORY_MAP.get(int(box.cls), int(box.cls))
                        detection = {
                            "image_id": image_id,
                            "category_id": category_id,
                            "bbox": [float(x1), float(y1), width, height],
                            "score": float(box.conf)
                        }
                        f.write(json.dumps(detection) + '\n')

                del results
                gc.collect()
        print("✅ Dự đoán hoàn tất!")

    # --- D. Tạo file submission cuối cùng ---
    print("\n🔄 Tổng hợp file submission.json...")
    all_detections = []
    with open(TEMP_OUTPUT_FILE, 'r') as f:
        for line in f:
            try:
                all_detections.append(json.loads(line))
            except json.JSONDecodeError:
                continue

    with open(SUBMISSION_FILE_PATH, 'w') as f:
        json.dump(all_detections, f, indent=4)

    print(f"🎉 Hoàn tất! File submission được tạo tại: {SUBMISSION_FILE_PATH}")

